This notebook is for a NN experiment on curves with size of the Tate-Shafarevich group equal to 4 and 9 including BSD features and the first 100 ap coefficients, while removing one feature at a time. The experiment is done on the original data.

In [1]:
from lib import utils, models, executor
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# fix the seed for reproducibility
seed = 42

# 1. Create balanced dataset of elliptic curves with size of the Tate-Shafarevich group equal to 4 and 9 containing all BSD features and the first 100 ap coefficients.

In [2]:
# load your data here. The following ensure this will work on Windows as well as Unix
path = Path("..") / "data_files" / "sha"/ "ecq_sha_B_100_conds_1_500000_reg.parquet"
df = utils.load_data(path)

Loaded the dataset with 120 features and 3064705 curves..


In [3]:
len_9 = df[df['sha'] == 9].shape[0]
df_balanced = df[df['sha'] == 4].sample(len_9, random_state=seed) 
df_balanced = pd.concat([df_balanced, df[df['sha'] == 9]])
df_balanced.sha.value_counts()

sha
4    50428
9    50428
Name: count, dtype: int64

In [4]:
bsd_features = ['special_value', 'torsion', 'real_period', 'regulator', 'tamagawa_product', 'sha']
primes=['2', '3', '5', '7', '11', '13', '17', '19', '23', '29', '31', '37',
       '41', '43', '47', '53', '59', '61', '67', '71', '73', '79', '83', '89',
       '97', '101', '103', '107', '109', '113', '127', '131', '137', '139',
       '149', '151', '157', '163', '167', '173', '179', '181', '191', '193',
       '197', '199', '211', '223', '227', '229', '233', '239', '241', '251',
       '257', '263', '269', '271', '277', '281', '283', '293', '307', '311',
       '313', '317', '331', '337', '347', '349', '353', '359', '367', '373',
       '379', '383', '389', '397', '401', '409', '419', '421', '431', '433',
       '439', '443', '449', '457', '461', '463', '467', '479', '487', '491',
       '499', '503', '509', '521', '523', '541']

In [5]:
df_balanced_bsd = df_balanced[bsd_features + primes].copy()

In [6]:
df_balanced_bsd.head(5)

,special_value,torsion,real_period,regulator,tamagawa_product,sha,2,3,5,7,...,467,479,487,491,499,503,509,521,523,541
334625,2.19751,2,0.54938,1.0,4,4,0.35355,0.00000,-0.44721,0.18898,...,0.27765,0.18276,-0.18126,0.72207,0.08953,0.57964,0.44324,-0.13143,0.61218,0.47293
1086182,3.22805,1,0.80701,1.0,1,4,0.70711,0.00000,0.00000,-0.56695,...,0.18510,-0.15992,-0.54377,-0.04513,0.62673,0.66882,0.39892,-0.59145,-0.02186,-0.34395
1782926,3.98612,2,0.49826,1.0,8,4,0.00000,0.57735,-0.44721,0.00000,...,-0.50902,0.18276,0.09063,0.27078,-0.71626,-0.17835,0.66486,0.30668,0.34982,0.64490
2484030,2.99537,1,0.09361,1.0,8,4,-0.35355,0.00000,0.00000,0.18898,...,0.34706,0.68537,0.29454,0.11282,0.24621,0.31211,0.59838,-0.37239,0.48100,-0.81687
3053287,2.23394,1,0.09308,1.0,6,4,-0.35355,-0.28868,0.00000,0.94491,...,-0.83294,-0.13707,0.90629,0.18052,-0.62673,0.98093,0.17730,0.48192,0.89640,-0.17197


# 2. Delete one BSD feature at a time (on original data).

The models achieve accuracy over 90% when removing one of the following: special value, torsion, and regulator.

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
# Neural Net classifier

# choose model parameters
hidden_units = [128,64,32]

# check if we have cuda available
device = 'cuda'

# Map labels to range starting from 0
df_BSD_label_mapped = df_balanced_bsd.copy()

# Neural net training for binary classification expects labels to start at 0
df_BSD_label_mapped['sha'] = df_BSD_label_mapped['sha'].map({4: 0, 9: 1})

# choose training parameters
loss_func = nn.CrossEntropyLoss()
num_epochs = 100
lr = 0.001
evaluator = accuracy_score

# Initialize an empty DataFrame to store the results
results_df_nn = pd.DataFrame(columns=['Feature Deleted', 'Accuracy'])

for i in range(len(bsd_features)-1):
    print(f'Running model without {bsd_features[i]}..')
    df_sub = df_BSD_label_mapped.drop(columns=[bsd_features[i]])
    input_dim, output_dim = utils.get_input_output_dim(df_sub, 'sha', if_regression=False)
    model = models.VanillaNN(input_dim, hidden_units, output_dim, if_dropout=False, dropout_rate=0.3, if_batchnorm=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr) # reinitialize optimizer
    train_dataloader, val_dataset, test_dataset = utils.prepare_data(df_sub, 'sha', device, if_regression=False, random_state=seed)
    model, train_eval_hist, val_eval_hist, train_loss_hist, val_loss_hist = executor.train(model, train_dataloader, val_dataset, loss_func, evaluator, optimizer, num_epochs, if_regression=False, verbose=False)
    acc = executor.test(model, test_dataset, evaluator, if_regression=False)
    print(f"Test accuracy: {acc:0.3f}")
    print('----------------------------------')
    
   # Append the results to the DataFrame
    results_df_nn = pd.concat([results_df_nn, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)

Running model without special_value..
The input dimension is 104 and the output dimension is 2.


Save the model from epoch 24 with Training CrossEntropyLoss : 0.0692, Validation CrossEntropyLoss : 0.1201. Training accuracy_score : 0.9732, Validation accuracy_score : 0.9533, to ../trained_models/model.pth.
Test accuracy: 0.949
----------------------------------
Running model without torsion..
The input dimension is 104 and the output dimension is 2.


/tmp/ipykernel_789/125803182.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_nn = pd.concat([results_df_nn, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)


Save the model from epoch 70 with Training CrossEntropyLoss : 0.0740, Validation CrossEntropyLoss : 0.1337. Training accuracy_score : 0.9749, Validation accuracy_score : 0.9595, to ../trained_models/model.pth.
Test accuracy: 0.957
----------------------------------
Running model without real_period..
The input dimension is 104 and the output dimension is 2.


Save the model from epoch 7 with Training CrossEntropyLoss : 0.5533, Validation CrossEntropyLoss : 0.6082. Training accuracy_score : 0.7043, Validation accuracy_score : 0.6516, to ../trained_models/model.pth.
Test accuracy: 0.656
----------------------------------
Running model without regulator..
The input dimension is 104 and the output dimension is 2.


Save the model from epoch 26 with Training CrossEntropyLoss : 0.0381, Validation CrossEntropyLoss : 0.0581. Training accuracy_score : 0.9836, Validation accuracy_score : 0.9770, to ../trained_models/model.pth.
Test accuracy: 0.974
----------------------------------
Running model without tamagawa_product..
The input dimension is 104 and the output dimension is 2.


Save the model from epoch 6 with Training CrossEntropyLoss : 0.4488, Validation CrossEntropyLoss : 0.5432. Training accuracy_score : 0.7930, Validation accuracy_score : 0.7247, to ../trained_models/model.pth.
Test accuracy: 0.725
----------------------------------


In [9]:
print(results_df_nn)

    Feature Deleted  Accuracy
0     special_value  0.949286
1           torsion  0.956524
2       real_period  0.656107
3         regulator  0.974321
4  tamagawa_product  0.725411
